In [ ]:
import pandas as pd
import numpy as np
!pip install rouge
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate
from tensorflow.keras.optimizers import Adam
!pip install nltk
import nltk
nltk.download('punkt')

from rouge import Rouge


# Define hyperparameters
vocab_size = 10000
embedding_dim = 300
hidden_dim = 256
dropout_rate = 0.2
learning_rate = 0.001
batch_size = 32
num_epochs = 10


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Load your dataset
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
import nltk
from nltk.tokenize import word_tokenize

# Define the number of examples to process
num_examples = 100  # Change this to the desired number of examples

# Load your dataset from Google Drive (only a subset)
file_path = '/content/drive/My Drive/train.csv'
train_data = pd.read_csv(file_path, nrows=num_examples)
file_path1 = '/content/drive/My Drive/test.csv'
test_data = pd.read_csv(file_path1, nrows=num_examples)
file_path2 = '/content/drive/My Drive/validation.csv'
val_data = pd.read_csv(file_path2, nrows=num_examples)

# Clean and preprocess the text
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    words = word_tokenize(text)  # Tokenize using NLTK
    cleaned_text = ' '.join(words)
    return cleaned_text

# Apply preprocessing to each dataset
def preprocess_dataset(dataset):
    dataset['cleaned_text'] = dataset['article'].apply(preprocess_text)
    dataset['cleaned_highlights'] = dataset['highlights'].apply(preprocess_text)

# Preprocess the train dataset
preprocess_dataset(train_data)

# Preprocess the validation dataset
preprocess_dataset(val_data)

# Preprocess the test dataset
preprocess_dataset(test_data)

# Save the preprocessed data to new CSV files
train_data.to_csv('preprocessed_train.csv', index=False)
val_data.to_csv('preprocessed_validation.csv', index=False)
test_data.to_csv('preprocessed_test.csv', index=False)

# Print the first few rows of each dataset to verify
print(train_data.head())
print(val_data.head())
print(test_data.head())


Mounted at /content/drive
                                         id  \
0  0001d1afc246a7964130f43ae940af6bc6c57f01   
1  0002095e55fcbd3a2f366d9bf92a95433dc305ef   
2  00027e965c8264c35cc1bc55556db388da82b07f   
3  0002c17436637c4fe1837c935c04de47adb18e9a   
4  0003ad6ef0c37534f80b55b4235108024b407f0b   

                                             article  \
0  By . Associated Press . PUBLISHED: . 14:11 EST...   
1  (CNN) -- Ralph Mata was an internal affairs li...   
2  A drunk driver who killed a young woman in a h...   
3  (CNN) -- With a breezy sweep of his pen Presid...   
4  Fleetwood are the only team still to have a 10...   

                                          highlights  \
0  Bishop John Folda, of North Dakota, is taking ...   
1  Criminal complaint: Cop used his role to help ...   
2  Craig Eccleston-Todd, 27, had drunk at least t...   
3  Nina dos Santos says Europe must be ready to a...   
4  Fleetwood top of League One after 2-0 win at S...   

                 

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, Concatenate
from tensorflow.keras.optimizers import Adam
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

# Load your preprocessed data (assuming CSV files are available)
# Use a very small subset of the data by specifying the number of examples
num_examples = 10  # Change this to the desired number of examples
train_data = pd.read_csv('preprocessed_train.csv', nrows=num_examples)
val_data = pd.read_csv('preprocessed_validation.csv', nrows=num_examples)
test_data = pd.read_csv('preprocessed_test.csv', nrows=num_examples)

# Define hyperparameters for a very small dataset
vocab_size = 2000  # Reduce vocabulary size
embedding_dim = 50  # Reduce embedding dimension
hidden_dim = 64  # Reduce hidden dimension
dropout_rate = 0.2
learning_rate = 0.001
batch_size = 4  # Reduce batch size for a very small dataset
num_epochs = 5  # You can reduce the number of epochs as needed



# Define the model with adjusted hyperparameters
encoder_inputs = Input(shape=(None,))
encoder_embed = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(hidden_dim, dropout=dropout_rate, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embed)

decoder_inputs = Input(shape=(None,))
decoder_embed = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(hidden_dim, dropout=dropout_rate, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embed, initial_state=[state_h, state_c])

attention = Attention(use_scale=True, dropout=dropout_rate)
context_vector = attention([decoder_outputs, encoder_outputs])

decoder_combined_context = Concatenate(axis=-1)([context_vector, decoder_outputs])

pointer_probs = Dense(1, activation='sigmoid')(decoder_combined_context)
generator_probs = Dense(vocab_size, activation='softmax')(decoder_combined_context)

final_probs = pointer_probs * generator_probs

model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=final_probs)

optimizer = Adam(learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy')



In [15]:
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge
import random

# Load your test data
val_data = pd.read_csv('preprocessed_validation.csv')
test_data = pd.read_csv('preprocessed_test.csv')

# Define your tokenizer and max text length (use the same values as during training)
# tokenizer = ...  # Define your tokenizer here
max_text_length = 100
max_summary_length = 50

# Load your trained model
model = tf.keras.models.load_model('my_trained_summarization_model.h5')

# Function to generate summaries
def generate_summary(input_sequence):
    if input_sequence is None or not isinstance(input_sequence, str) or len(input_sequence) == 0:
        return ""  # Return an empty summary if input_sequence is None, not a string, or an empty string

    # Tokenize the input sequence
    input_sequence = tokenizer.texts_to_sequences([input_sequence])

    # If '<start>' token is in the word_index, use it; otherwise, use 0 as a placeholder
    if '<start>' in tokenizer.word_index:
        start_token = tokenizer.word_index['<start>']
    else:
        start_token = 0

    # If '<end>' token is in the word_index, use it; otherwise, use 0 as a placeholder
    if '<end>' in tokenizer.word_index:
        end_token = tokenizer.word_index['<end>']
    else:
        end_token = 0

    # Pad the input sequence
    if isinstance(input_sequence, str) and len(input_sequence) > 0:
        input_sequence = pad_sequences([input_sequence], maxlen=max_text_length, padding='post')
    else:
        input_sequence = np.zeros((1, max_text_length))  # Create a padded sequence of zeros for empty or non-string inputs

    # Initialize the target sequence with the start token
    target_sequence = np.zeros((1, max_summary_length))
    target_sequence[0, 0] = start_token

    # Generate the summary one word at a time
    for i in range(1, max_summary_length):
        # Predict the next word
        predictions = model.predict([input_sequence, target_sequence], batch_size=10000, verbose=0)

        # Sample the word with the highest probability
        sampled_token_index = np.argmax(predictions[0, i, :])
        target_sequence[0, i] = sampled_token_index

        # If the generated word is the end token or the placeholder, stop generating
        if sampled_token_index == end_token or sampled_token_index == 0:
            break

    # Convert the generated sequence back to text
    generated_summary = " ".join([tokenizer.index_word.get(int(token), '<unk>') for token in target_sequence[0] if token != 0])
    return generated_summary  # Return the generated summary

# Number of random samples to evaluate (adjust as needed)
num_samples_to_evaluate = 100

# Randomly select a subset of samples for evaluation
sample_indices = random.sample(range(len(test_data)), num_samples_to_evaluate)
sample_data = test_data.iloc[sample_indices]

# Lists to store generated and reference summaries
generated_summaries = []
reference_summaries = []

# Generate and collect summaries
for input_seq in sample_data['article']:
    if isinstance(input_seq, str) and len(input_seq) > 0:
        generated_summary = generate_summary(input_seq)
        generated_summaries.append(generated_summary)
        reference_summaries.append(sample_data.loc[sample_data['article'] == input_seq, 'cleaned_highlights'].values[0])

# Calculate BLEU scores
bleu_scores = corpus_bleu(reference_summaries, generated_summaries)

# Calculate ROUGE scores
rouge = Rouge()
rouge_scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)


# Print the evaluation results
print("Average BLEU Score:", np.mean(bleu_scores))
print("ROUGE Scores:", rouge_scores)


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU Score: 6.164164564713577e-232
ROUGE Scores: {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}
